# Intro

Competition home page: https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/overview

# Import library

In [ ]:
# Install detecton2
# No pre-build for tocrch 1.9.1 and cuda 11.0. Consider cpu instead.
#!pip install detectron2 -f \
#  https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.9/index.html

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import os
import copy
import pickle
import argparse
import json
import random
import sys
import time
import datetime
import logging


import pandas as pd
import numpy as np

from PIL import Image
import cv2
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold

import torch
from detectron2 import model_zoo
from detectron2.structures import BoxMode
import detectron2.data.transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator, PascalVOCDetectionEvaluator
from detectron2.config import CfgNode as CN
from detectron2.config import get_cfg
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.logger import setup_logger, log_every_n_seconds
from detectron2.utils.visualizer import Visualizer
from detectron2.engine.hooks import HookBase
import detectron2.utils.comm as comm



# Set configs

In [ ]:
thingClasses = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis",
    "No finding"
]

cfgDict = {
    "dicomPath": "../input/vinbigdata-chest-xray-abnormalities-detection/train/",
    "orgDataPath": "../input/vinbigdata-chest-xray-abnormalities-detection/",
    "newDataPath": "../input/vinbigdata-chest-xray-resized-png-256x256/",
    "cachePath": "./",
    "trainDataName": "vinbigdataTrain",
    "validDataName": "vinbigdataValid",
    "sampleSize": 1000,
    "imSize": 256,
    "modelName": "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",
    "debug": False,
    "outdir": "./results/",
    "logFile": "log.txt",
    "splitMode": True,
    "seed": 111,
    "device": "cuda",
    "iter": 1000,
    "ims_per_batch": 16,
    "roi_batch_size_per_image": 512,
    "eval_period": 20,
    "lr_scheduler_name": "WarmupCosineLR",
    "base_lr": 0.001,
    "checkpoint_period":500,
    "num_workers": 4,
    "score_thresh_test": 0.05,
    "augKwargs": {
        "RandomFlip": {"prob": 0.5},
        "RandomRotation": {"angle": [0,360]}
    }
}

setup_logger(os.path.join(cfgDict["outdir"],cfgDict["logFile"]))

# Preprocess data

We use the dataset [VinBigData Chest X-ray Resized PNG (256x256)](https://www.kaggle.com/xhlulu/vinbigdata-chest-xray-resized-png-256x256).

Example code is provided below.

In [ ]:
dicomPath = cfgDict["dicomPath"]
for file in os.listdir(dicomPath):
    filePath = os.path.join(dicomPath,file)
    break
# Read dicom
dicom = pydicom.read_file(filePath)
# Transform raw image to "human-friendly" view
imgArray = apply_voi_lut(dicom.pixel_array,dicom)
# Fix inverted image
if dicom.PhotometricInterpretation == "MONOCHROME1":
    imgArray = np.amax(imgArray) - imgArray
# Scale image value to (0,255)        
imgArray = imgArray - np.min(imgArray)
imgArray = imgArray / np.max(imgArray)
imgArray = (imgArray * 255).astype(np.uint8)
# Resize image to (imSize x imSize)
imSize = 256
im = Image.fromarray(imgArray)
im = im.resize((imSize,imSize),Image.LANCZOS)
# Display image
im

# Prepare datasets

In [ ]:
def getDatasetDicts(cfg,dfTrain,dfMeta,dataType="train",imSize=256,cache=False):
    """Function to create dataset dicts"""
    
    cachePath = os.path.join(cfg["cachePath"],"cache"+dataType+".pkl")
    datasetDicts = []
    
    if not cache and os.path.exists(cachePath):
        # Load dicts
        with open(cachePath, mode="rb") as f:
            datasetDicts = pickle.load(f)
    else:
        # Cache dicts
        for index,metaRow in dfMeta.iterrows():
            datasetDict = {}
            annotations = []

            imageId,h,w = metaRow.values
            if dataType.lower() != "test":
                filename = os.path.join(newDataPath,"train",imageId+".png")
            else:
                filename = os.path.join(newDataPath,"test",imageId+".png")
            datasetDict["file_name"] = filename
            datasetDict["image_id"] = imageId
            datasetDict["height"] = imSize
            datasetDict["width"] = imSize
            
            # Add annotations for training/validation data
            if dataType.lower() != "test":
                for index2,row in dfTrain[dfTrain["image_id"]==imageId].iterrows():
                    classId = row["class_id"]          
                    if classId != 14:
                        hRatio = imSize/h
                        wRatio = imSize/w
                        bboxResized = [ float(row["x_min"])*wRatio,
                                        float(row["y_min"])*hRatio,
                                        float(row["x_max"])*wRatio,
                                        float(row["y_max"])*hRatio ]
                    else: 
                        bboxResized = [0, 0, imSize, imSize]      
                    annotation = { "bbox": bboxResized,
                                    "bbox_mode": BoxMode.XYXY_ABS,
                                    "category_id": classId }
                    annotations.append(annotation)
                datasetDict["annotations"] = annotations
            
            datasetDicts.append(datasetDict)

        with open(cachePath, mode="wb") as f:
            pickle.dump(datasetDicts, f)
    
    return datasetDicts


# Prepare augmentation

In [ ]:
class AugMapper:
    """Custom mapper class for augmentations"""

    def __init__(self, cfg, isTrain=True):
        augKwargs = cfg["augKwargs"]
        augList = []
        # Define a sequence of augmentations
        if isTrain:
            augList.extend([getattr(T, name)(**kwargs) for name, kwargs in augKwargs.items()])
        self.augmentations = T.AugmentationList(augList)
        self.isTrain = isTrain

    def __call__(self, datasetDict):
        datasetDict = copy.deepcopy(datasetDict)  # it will be modified by code below
        image = utils.read_image(datasetDict["file_name"], format="BGR")
        augInput = T.AugInput(image) # the augmentation input
        transforms = self.augmentations(augInput) # apply the augmentation
        image = augInput.image # new image
        imShape = image.shape[:2]  # h, w
        datasetDict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32")) # HWC to CHW
        annos = [ utils.transform_instance_annotations(annotation, transforms, imShape) 
                    for annotation in datasetDict.pop("annotations") 
                    if annotation.get("iscrowd", 0) == 0 ] # apply the augmentation to annotation
        instances = utils.annotations_to_instances(annos, imShape)
        datasetDict["instances"] = utils.filter_empty_instances(instances)
        
        return datasetDict

# Prepare loss eval hook for validation

In [ ]:
class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        comm.synchronize()

        return mean_loss
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            mean_loss = self._do_loss_eval()
            self.trainer.storage.put_scalars(validation_loss=mean_loss)
            print("validation do loss eval", mean_loss)


# Custom DefaultTrainer

In [ ]:
class MyTrainer(DefaultTrainer):
    """Overwrite DefaultTrainer methods"""
    
    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
            cfg, mapper=AugMapper(cfg, True), sampler=sampler
        )

    @classmethod
    def build_test_loader(cls, cfg, datasetName):
        return build_detection_test_loader(
            cfg, datasetName, mapper=AugMapper(cfg, False)
        )

    @classmethod
    def build_evaluator(cls, cfg, datasetName, outputFolder=None):
        if outputFolder is None:
            outputFolder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(datasetName, ("bbox",), False, output_dir=outputFolder)
    
    def build_hooks(self):
        hooks = super(MyTrainer, self).build_hooks()
        cfg = self.cfg
        if len(cfg.DATASETS.TEST) > 0:
            loss_eval_hook = LossEvalHook(
                cfg.TEST.EVAL_PERIOD,
                self.model,
                MyTrainer.build_test_loader(cfg, cfg.DATASETS.TEST[0]),
            )
            hooks.insert(-1, loss_eval_hook)

        return hooks

# Load, split, and register data

In [ ]:
orgDataPath = cfgDict["orgDataPath"]
newDataPath = cfgDict["newDataPath"]
trainCSVPath = os.path.join(orgDataPath,"train.csv")
trainMetaCSVPath = os.path.join(newDataPath,"train_meta.csv")

dfTrain = pd.read_csv(trainCSVPath)
dfTrainMeta = pd.read_csv(trainMetaCSVPath)
dfTrainMeta = dfTrainMeta[:cfgDict["sampleSize"]]

In [ ]:
datasetDicts = getDatasetDicts(cfgDict,dfTrain=dfTrain,dfMeta=dfTrainMeta,cache=True)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=cfgDict["seed"])
y = np.array([int(len(d["annotations"]) > 0) for d in datasetDicts])
splitIdx = list(skf.split(datasetDicts, y))
trainIdx, validIdx = splitIdx[0]

In [ ]:
DatasetCatalog.clear()
DatasetCatalog.register(
        cfgDict["trainDataName"],
        lambda: getDatasetDicts(cfgDict,dfTrain=dfTrain,dfMeta=dfTrainMeta.iloc[trainIdx],dataType="train",cache=True)
    )
MetadataCatalog.get(cfgDict["trainDataName"]).set(thing_classes=thingClasses)
DatasetCatalog.register(
        cfgDict["validDataName"],
        lambda: getDatasetDicts(cfgDict,dfTrain=dfTrain,dfMeta=dfTrainMeta.iloc[validIdx],dataType="valid",cache=True)
    )
MetadataCatalog.get(cfgDict["validDataName"]).set(thing_classes=thingClasses)

# Visualize data

In [ ]:
vinbigdataMetadata = MetadataCatalog.get(cfgDict["validDataName"])
d = datasetDicts[3]
img = cv2.imread(d["file_name"])
visualizer = Visualizer(img[:, :, ::-1], metadata=vinbigdataMetadata, scale=1.5)
out = visualizer.draw_dataset_dict(d)
Image.fromarray(out.get_image()[:, :, ::-1])

# Create Yacs config

In [ ]:
cfg = get_cfg()

cfg.augKwargs = CN(cfgDict["augKwargs"])  # pass augKwargs to cfg as a CN
cfg.merge_from_file(model_zoo.get_config_file(cfgDict["modelName"]))
cfg.MODEL.DEVICE = cfgDict["device"]
cfg.OUTPUT_DIR = cfgDict["outdir"]
cfg.DATASETS.TRAIN = (cfgDict["trainDataName"],)
if cfgDict["splitMode"] is None:
    cfg.DATASETS.TEST = ()
else:
    cfg.DATASETS.TEST = (cfgDict["validDataName"],)
    cfg.TEST.EVAL_PERIOD = cfgDict["eval_period"]
cfg.DATALOADER.NUM_WORKERS = cfgDict["num_workers"]
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(cfgDict["modelName"])
cfg.SOLVER.IMS_PER_BATCH = cfgDict["ims_per_batch"]
cfg.SOLVER.LR_SCHEDULER_NAME = cfgDict["lr_scheduler_name"]
cfg.SOLVER.BASE_LR = cfgDict["base_lr"]
cfg.SOLVER.MAX_ITER = cfgDict["iter"]
cfg.SOLVER.CHECKPOINT_PERIOD = cfgDict["checkpoint_period"]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = cfgDict["roi_batch_size_per_image"]
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(thingClasses)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Train model

In [ ]:
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Evaluation

In [ ]:
dfMetrics = pd.read_json(os.path.join(cfgDict["outdir"],"metrics.json"), orient="records", lines=True)
dfMetrics = dfMetrics.sort_values("iteration")
dfMetrics.head()

In [ ]:
dfTrainLoss = dfMetrics[~dfMetrics["total_loss"].isna()]
plt.plot(dfTrainLoss["iteration"], dfTrainLoss["total_loss"], c="C0", label="train")
if "validation_loss" in dfMetrics.columns:
    dfValidLoss = dfMetrics[~dfMetrics["validation_loss"].isna()]
    plt.plot(dfValidLoss["iteration"], dfValidLoss["validation_loss"], c="C1", label="validation")

plt.legend()
plt.title("Loss curve")
plt.xlabel("Iteration")
plt.show()

# Predict

In [ ]:
# Same cfg from trainer and use the final model output to initialize the predictor
cfg.MODEL.WEIGHTS = os.path.join(cfgDict["outdir"],"model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfgDict["score_thresh_test"]
predictor = DefaultPredictor(cfg)

d = datasetDicts[3]
im = cv2.imread(d["file_name"])
if predictor.input_format == "RGB":
    im = im[:, :, ::-1]
height, width = im.shape[:2]
image = torch.as_tensor(im.astype("float32").transpose(2, 0, 1))
inputs = [{"image": image, "height": height, "width": width}]
outputs = predictor.model(inputs)
output = outputs[0]

visualizer = Visualizer(im,metadata=vinbigdataMetadata, scale=1.5)
out = visualizer.draw_instance_predictions(output["instances"].to("cpu"))
Image.fromarray(out.get_image()[:, :, ::-1])

# Reference

https://www.kaggle.com/corochann/vinbigdata-detectron2-train#Customizing-detectron2-trainer

https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image

https://detectron2.readthedocs.io/en/latest/tutorials

https://eidos-ai.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e